In [ ]:
import os
import string
import numpy as np
import configparser

from datetime import date
from pytube import YouTube
from moviepy.editor import VideoFileClip

#### Download Youtube video and extract the audio in .wav

In [ ]:
def path_url():
    arq = configparser.RawConfigParser()
    arq.read('config.txt')

    # Get the video link and save path
    video_url =  arq.get('URLS','url')
    save_path = arq.get('Path','save_path')

    today_date = date.today().strftime('%d%m%Y')

    sub_save_path = save_path + today_date + '/'
    print('Link and Path were collected\n')
    
    return video_url, sub_save_path

def aud_ext(sub_save_path, rename_filename):  # Extract .wav file from a .mp4 or 3gp file  
    video_path = os.path.join(sub_save_path, f'{rename_filename}') 
    output_path = os.path.join(sub_save_path, f'{rename_filename[:-5]}.wav')

    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_path, codec='pcm_s16le')
    print('The .wav file was saved with success!')
    
def main():
    video_url, sub_save_path = path_url()

    youtube = YouTube(video_url)
    print('passed')

    stream = youtube.streams.first()
    print('two')

    stream.download(sub_save_path)
    print('The video was downloaded')

    #Rename the downloaded file
    original_filename = stream.default_filename
    
    # Remove any problematic characters from the video title
    ## First cleaning step
    valid_chars = "-.’'_() %s%s" % (string.ascii_letters, string.digits)
    original_filename = ''.join(c for c in original_filename if c in valid_chars)
    new_filename = original_filename.replace(" ","")

    ## Second step
    valid_chars = "_. %s%s" % (string.ascii_letters, string.digits)
    new_filename = ''.join(c for c in new_filename if c in valid_chars)


    rename_filename = original_filename.replace(original_filename, new_filename)

    rename_filename = os.path.join(sub_save_path, rename_filename)
    original_save = os.path.join(sub_save_path, original_filename)

    os.rename(original_save,  rename_filename)

    print('\nThe file was renamed and The Audio extraction starts')
    aud_ext(sub_save_path, rename_filename)



In [ ]:
main()

#### Separete the vocal from the rst of the sounds in a .wav file

In [ ]:
import librosa
import numpy as np

from scipy.io import wavfile

In [ ]:
aud_file = ''

In [ ]:
audio,sr = librosa.load(aud_file)

In [ ]:
# Compute a magnitude spectrogram
spec = np.abs(librosa.stft(audio))

In [ ]:
# Convert the magnitude spectrogram to decibels (dB)
spec_db = librosa.amplitude_to_db(spec, ref=np.max)


In [ ]:
# Estimate the vocals using a pre-trained model or signal processing techniques
# Example: Use harmonic-percussive source separation (HPSS)
harmonic, percussive = librosa.effects.hpss(spec_db)


In [ ]:
# Invert the sepatated vocals back to the time domain
vocals = librosa.istft(harmonic)

In [ ]:
# Rescale the audio to 16 bit integer range
vocal_int = (vocals*(2**15-1)).astype(np.int16)

In [ ]:
# Output
out = ''
wavfile.write(out, sr, vocal_int)

In [ ]:
from pydub import AudioSegment
import os

In [ ]:
def convert_wav_to_mp3(wav_file, mp3_file):
    # Load WAV file
    audio = AudioSegment.from_wav(wav_file)
    
    # Export as MP3 file
    audio.export(mp3_file, format='mp3')


In [ ]:
wav_file = '/Vocal_Extraction/Results/Vid_Aud/28052023/aud.wav'
mp3_file = 'Vocal_Extraction/Results/Vid_Aud/28052023/aud.mp3'
convert_wav_to_mp3(wav_file, mp3_file)
